In [1]:
import re
import keras
import numpy as np

from keras.layers import Dense, LSTM
from collections import defaultdict

In [2]:
text = open('AdamsText.txt', 'r', encoding='utf-8').read()

In [3]:
def reformat(stroke):
    stroke = stroke.lower()
    stroke = re.sub(r'[ё]', 'е', stroke)
    stroke = re.sub(r"[^а-я\.\-\s+]+", "", stroke)
    stroke = re.sub(r'\s+', ' ', stroke)
    stroke = re.sub(r'(\.+\s*)+', '. ', stroke)
    stroke = re.sub(r'\s*\.+\s*', '. ', stroke)
    return stroke

In [4]:
text = reformat(text)
chars_list = sorted(list(set(text)))

In [5]:
chars = dict((c, i) for i, c in enumerate(chars_list))
indexes = dict((i, c) for i, c in enumerate(chars_list))
print(chars_list)
print(chars)
print(indexes)

[' ', '-', '.', 'а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я']
{' ': 0, '-': 1, '.': 2, 'а': 3, 'б': 4, 'в': 5, 'г': 6, 'д': 7, 'е': 8, 'ж': 9, 'з': 10, 'и': 11, 'й': 12, 'к': 13, 'л': 14, 'м': 15, 'н': 16, 'о': 17, 'п': 18, 'р': 19, 'с': 20, 'т': 21, 'у': 22, 'ф': 23, 'х': 24, 'ц': 25, 'ч': 26, 'ш': 27, 'щ': 28, 'ъ': 29, 'ы': 30, 'ь': 31, 'э': 32, 'ю': 33, 'я': 34}
{0: ' ', 1: '-', 2: '.', 3: 'а', 4: 'б', 5: 'в', 6: 'г', 7: 'д', 8: 'е', 9: 'ж', 10: 'з', 11: 'и', 12: 'й', 13: 'к', 14: 'л', 15: 'м', 16: 'н', 17: 'о', 18: 'п', 19: 'р', 20: 'с', 21: 'т', 22: 'у', 23: 'ф', 24: 'х', 25: 'ц', 26: 'ч', 27: 'ш', 28: 'щ', 29: 'ъ', 30: 'ы', 31: 'ь', 32: 'э', 33: 'ю', 34: 'я'}


In [6]:
seq_len = 12
dataX = list(s[i:i + seq_len] for s in text.split('.') for i in range(len(s) - seq_len))
dataY = list(s[seq_len + i] for s in text.split('.') for i in range(len(s) - seq_len))
print(dataX[:10])
print(dataY[:10])

['глава каждое', 'лава каждое ', 'ава каждое у', 'ва каждое ут', 'а каждое утр', ' каждое утро', 'каждое утро ', 'аждое утро т', 'ждое утро тр', 'дое утро тра']
[' ', 'у', 'т', 'р', 'о', ' ', 'т', 'р', 'а', 'д']


### Приведем данные к OneHot

In [7]:
X = np.zeros((len(dataX), seq_len, len(chars)))
Y = np.zeros((len(dataX), len(chars)))
for i, s in enumerate(dataX):
    for j, ch in enumerate(s):
        X[i, j, chars[ch]] = 1
    Y[i, chars[dataY[i]]] = 1

### LSTM

In [8]:
model = keras.Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(Y.shape[1], activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy')
model.fit(X, Y, epochs=30)
model.summary()

Epoch 1/30
964/964 [==============================] - 44s 42ms/step - loss: 2.7945
Epoch 2/30
964/964 [==============================] - 45s 46ms/step - loss: 2.4131
Epoch 3/30
964/964 [==============================] - 41s 42ms/step - loss: 2.2531
Epoch 4/30
964/964 [==============================] - 38s 39ms/step - loss: 2.1232
Epoch 5/30
964/964 [==============================] - 37s 39ms/step - loss: 1.9982
Epoch 6/30
964/964 [==============================] - 36s 37ms/step - loss: 1.8750
Epoch 7/30
964/964 [==============================] - 36s 38ms/step - loss: 1.7398
Epoch 8/30
964/964 [==============================] - 41s 43ms/step - loss: 1.5936
Epoch 9/30
964/964 [==============================] - 44s 45ms/step - loss: 1.4298
Epoch 10/30
964/964 [==============================] - 36s 38ms/step - loss: 1.2546
Epoch 11/30
964/964 [==============================] - 47s 49ms/step - loss: 1.0778
Epoch 12/30
964/964 [==============================] - 41s 43ms/step - loss: 0.8969
E

In [22]:
num_tests = 4
num_predict = 30

for _ in range(num_tests):
    cur_sentence = dataX[np.random.randint(len(dataX) - 1)]
    sentence = cur_sentence
    res = cur_sentence
    appended = ""
    for _ in range(num_predict):
        cur_hot = np.zeros((1, seq_len, len(chars_list)))
        for i, ch in enumerate(cur_sentence):
            cur_hot[0, i, chars[ch]] = 1
        predict = model.predict(cur_hot, verbose=0)
        next_ch = indexes[np.argmax(predict)]
        cur_sentence = cur_sentence[1:] + next_ch
        appended += next_ch
    print('Start sentence : ', sentence)
    print('Result of predict : ', res + appended, '\n')

Start sentence :  кличем внеза
Result of predict :  кличем внезапного восторга он спрыгнул с у 

Start sentence :   мебельные м
Result of predict :   мебельные магазины громят воскликнул форд 

Start sentence :  где вы ее на
Result of predict :  где вы ее нашли и как сюда переместили по- 

Start sentence :  е подробност
Result of predict :  е подробности аварии не имеют значения пос 



#### Готовим цепь

In [10]:
nexts = defaultdict(lambda: defaultdict(lambda: 0))
for s, c in zip(dataX, dataY):
    nexts[s][c] += 1
for sent in dataX:
    sum_next = sum([v for v in nexts[sent].values()])
    for next in nexts[sent]:
        nexts[sent][next] /= sum_next

### Протестим

In [23]:
for _ in range(num_tests):
    cur_sentence = dataX[np.random.randint(len(dataX) - 1)]
    sentence = cur_sentence
    appended = ""
    for _ in range(num_predict):
        next_symbols = nexts[cur_sentence]
        p = list(next_symbols.values())
        if len(p) == 0:
            break
        next_symb = list(next_symbols.items())[np.random.choice(len(p), p=np.array(p))][0]
        cur_sentence += next_symb
        cur_sentence = cur_sentence[1:]
        appended += next_symb
    print('Start sentence : ', sentence)
    print('Result of predict : ', sentence + appended, '\n')

Start sentence :  его лучи лас
Result of predict :  его лучи ласкали брикеты мороженого от чег 

Start sentence :  нь интересно
Result of predict :  нь интересно заявил форд пододвинув к арту 

Start sentence :   субэфирный 
Result of predict :   субэфирный чуткомат 

Start sentence :  ловно это бы
Result of predict :  ловно это был не чай а но в данном случае  

